In [42]:
import copy
import mujoco
from dm_control import mjcf
import numpy as np
import cv2
import scipy
from scipy.spatial.transform import Rotation as Rotater
import common
from dm_control.mujoco import Physics
from lxml import etree
from dm_control.suite import base
from dm_control.rl import control
from dm_control import viewer
import mediapy as media

np.set_printoptions(precision=5, suppress=True)

In [43]:

def compute_gains(model, data, configuration, Q, R):
    # 1. Set configuration and find control that stabilizes it (ctrl0)
    newdata = mujoco.MjData(model)
    newdata = copy.copy(data)

    mujoco.mj_resetData(model, newdata)
    newdata.qpos = configuration
    # compute the control
    mujoco.mj_forward(model, newdata)
    newdata.qacc = 0
    mujoco.mj_inverse(model, newdata)

    # define control and configuration to linearize around
    # print(newdata.qfrc_actuator)
    ctrl0 = newdata.qfrc_inverse.copy() @ np.linalg.pinv(newdata.actuator_moment)
    qpos0 = newdata.qpos.copy()
    print('ctrl0:\n', ctrl0)
    print('qpos0:\n', qpos0)

    # 2. Compute LQR gains given weights
    mujoco.mj_resetData(model, newdata)
    newdata.ctrl = ctrl0
    newdata.qpos = qpos0

    # 3. Allocate the A and B matrices, compute them.
    A = np.zeros((2 * model.nv, 2 * model.nv))
    B = np.zeros((2 * model.nv, model.nu))
    epsilon = 1e-6
    flg_centered = True
    mujoco.mjd_transitionFD(model, newdata, epsilon, flg_centered, A, B, None, None)
    

    #print A, B, Q, R in format 'A:\n{}\n'
    print('A:\n{}\nB:\n{}\nQ:\n{}\nR:\n{}'.format(A.shape, B.shape, Q.shape, R.shape))
    print('A:\n{}\nB:\n{}\nQ:\n{}\nR:\n{}'.format(A, B, Q, R))

    # return None, None

    # Solve discrete Riccati equation.
    P = scipy.linalg.solve_discrete_are(A, B, Q, R)
    print('P:\n{}'.format(P.shape))

    # Compute the feedback gain matrix K.
    K = np.linalg.inv(R + B.T @ P @ B) @ B.T @ P @ A

    #print P, K in format 'A:\n{}\n'
    print('P:\n{}\nK:\n{}'.format(P, K))

    return ctrl0, K

In [44]:
def simulate():

    CART_POLE_MJCF = "legwheel_robot2.xml" #"cartpole2.xml"
    sim_model = mujoco.MjModel.from_xml_path(CART_POLE_MJCF)
    sim_data = mujoco.MjData(sim_model)
    renderer = mujoco.Renderer(sim_model, height=480, width=640)

    # Make a new camera, move it to a closer distance.
    camera = mujoco.MjvCamera()
    mujoco.mjv_defaultFreeCamera(sim_model, camera)
    camera.lookat = [0, -10, 8]
    camera.azimuth = 90
    camera.elevation = -30
    camera.distance = 1
    q0 = copy.copy(sim_data.qpos)

    print('sim_model.nv:', sim_model.nv, 'sim_model.nu:', sim_model.nu, 'sim_model.nq:', sim_model.nq, 'sim_model.na:', sim_model.na)  
    print('sim_data.qpos:', sim_data.qpos, '\nsim_data.qvel:', sim_data.qvel, '\nsim_data.qacc:', sim_data.qacc, '\nsim_data.ctrl:', sim_data.ctrl)
    print('sim_data.site_xpos:', sim_data.site_xpos, '\nsim_data.site_xmat:', sim_data.site_xmat)

    # Parameters.
    DURATION = 10  # seconds
    BALANCE_STD = 0.2  # actuator units
    FRAMERATE = 30

    
    ang_euler = Rotater.from_quat([0, 0, 0, 1]).as_euler('xyz', degrees=True)
    ang_euler = [0, 0, 0]
    quat = Rotater.from_euler('xyz', ang_euler, degrees=True).as_quat()
    print('ang_euler:', ang_euler, 'quat:', quat)

    qpos0 = np.array([0., 0., 1.55*np.cos(ang_euler[1]*np.pi/180), quat[3], quat[0], quat[1], quat[2], 0, 0, 0, 0, 0, 0, 0., 0.]) #give initial pose
    target = np.array([0., 0., 1.54, 1., 0., 0., 0., 0., 0., 0, 0, 0, 0, 0., 0.]) #give target pose which is the stable pose

    Q = np.eye(sim_model.nv*2) * 1
    R = np.eye(sim_model.nu) * 1

    dq = np.zeros(sim_model.nv)
    # mujoco.mj_differentiatePos(sim_model, dq, 1, target, sim_data.qpos)
    # print('dq:', dq)
    # return

    # ctrl0, K = compute_gains(uncertain_model, uncertain_data, target)
    ctrl0, K = compute_gains(sim_model, sim_data, target, Q, R)
    # return

    #print('ctrl0:', ctrl0, 'K:', K)

    # Reset data, set initial pose.
    mujoco.mj_resetData(sim_model, sim_data)
    sim_data.qpos = qpos0

    qhist = []
    frames = []
    while sim_data.time < DURATION:
        # Get state difference dx.
        mujoco.mj_differentiatePos(sim_model, dq, 1, target, sim_data.qpos)
        dx = np.hstack((dq, sim_data.qvel)).T

        # LQR control law.
        sim_data.ctrl = ctrl0 - K @ dx + np.random.randn(sim_model.nu)
        #print('ctrl:', sim_data.ctrl)

        # Step the simulation.
        mujoco.mj_step(sim_model, sim_data)

        # Save history.
        if len(frames) < sim_data.time * FRAMERATE:
            qhist.append(sim_data.qpos.copy())
            #camera.lookat = sim_data.body('root').subtree_com
            renderer.update_scene(sim_data, camera)
            img = renderer.render()
            frames.append(img)
    print("show video:")
    media.show_video(frames, fps=FRAMERATE)
    
    
simulate()

sim_model.nv: 14 sim_model.nu: 4 sim_model.nq: 15 sim_model.na: 0
sim_data.qpos: [0.   0.   1.54 1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.  ] 
sim_data.qvel: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 
sim_data.qacc: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 
sim_data.ctrl: [0. 0. 0. 0.]
sim_data.site_xpos: [[0. 0. 0.]] 
sim_data.site_xmat: [[0. 0. 0. 0. 0. 0. 0. 0. 0.]]
ang_euler: [0, 0, 0] quat: [0. 0. 0. 1.]
ctrl0:
 [-2083.59336 -2083.59336     0.          0.     ]
qpos0:
 [0.   0.   1.54 1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.  ]
A:
(28, 28)
B:
(28, 4)
Q:
(28, 28)
R:
(4, 4)
A:
[[ 1.       0.      -0.00023 -0.       0.00001  0.      -0.00022  0.00001
   0.00004  0.      -0.00022  0.00001  0.00004  0.       0.002    0.
  -0.00001 -0.       0.      -0.      -0.00001  0.       0.       0.
  -0.00001  0.       0.       0.     ]
 [-0.       1.       0.       0.00085 -0.      -0.       0.00009 -0.00001
  -0.00001  0.      -0.00009  0.00001  0.00001  0.  

This browser does not support the video tag.
